In [4]:
process.chdir("G:\\src\\github.com\\ddspog\\my-spotify-top50");

In [261]:
require('jsdom-global')(`<!DOCTYPE html><body><div id="chart"></div></body></html>`, {
    beforeParse(window) {
        window.Element.prototype.getComputedTextLength = function() {
          return 60.0
        }
    }
});

site = () => document.documentElement.innerHTML;

d3 = require('d3');
fs = require('fs');
DisjointSet = require( 'aureooms-js-disjoint-set' ) ;
//{makeset, union, find} = DisjointSet.Forest;
shuffle = require('shuffle-array');
request = require('request');

console.log("Loaded DOM and packages.");

Loaded DOM and packages.


In [274]:
var spotifyToken = 'BQDi1fATKYurnKr-zUgamODDW0qvjBSjPrUz3MiuCHiUBaGAzMw8UBgyu-BS2kgOeZ7L-ZMQm2Rnd4Mv_SvQvvAvW6wAb46xxh4mST3QbOINaT_AJnkyV0VVxxHi-AZZyZZ0gd0Gof9oCqX3sUcsAPD8eLk';

In [275]:
var dataTop = {};

request({
    url: 'https://api.spotify.com/v1/me/top/artists?limit=50',
    headers: {
        'Accept': 'application/json',
        'Authorization': 'Bearer ' + spotifyToken
    }
}, function(error, res, body) {
    if (error) return console.log(error);
    
    dataTop = JSON.parse(body);
    
    console.log('Request Completed');
});

console.log('Request Processed');

Request Processed
Request Completed


In [276]:
mapTop = {};

dataTop.items.forEach(function(el, i, arr) {
    mapTop[el.id] = {
        name: el.name,
        genres: el.genres,
        img: el.images[0].url,
        url: el.external_urls.spotify,
        popularity: el.popularity
    }
});

In [278]:
for(id in mapTop) {
    request({
        url: 'https://api.spotify.com/v1/artists/' + id + '/related-artists',
        headers: {
            'Accept': 'application/json',
            'Authorization': 'Bearer ' + spotifyToken
        }
    }, ((reqID) => function(error, res, body) {
        if (error) return console.log(error);
    
        mapTop[reqID].relatedData = JSON.parse(body);
        
    })(id));
}

console.log('Request Processed');

Request Processed


In [279]:
for(id in mapTop){
    mapTop[id].related = {};
    
    mapTop[id].relatedData.artists.forEach(((relID) => function(el, i, arr) {
        mapTop[relID].related[el.id] = {
            name: el.name,
            genres: el.genres,
            popularity: el.popularity
        }
    })(id));    
}

In [299]:
graph = { nodes: [], edges: [] };

console.log('Start of graphing');

for (id in mapTop) {
    graph.nodes.push({
        id: id,
        name: mapTop[id].name,
        genres: mapTop[id].genres,
        img: mapTop[id].img,
        url: mapTop[id].url,
        popularity: mapTop[id].popularity
    });
}

for (artistID in mapTop) {
    for (relatedID in mapTop[artistID].related) {
        
        if(relatedID in mapTop) {
            
            graph.edges.push({
                source: artistID,
                target: relatedID,
                popularity: mapTop[artistID].popularity * mapTop[artistID].related[relatedID].popularity,
                type: 'related'
            });
        }
    }
}

genresRelation = {};

for (i in mapTop) {
    for (j in mapTop) {
        if (i != j) {
            gA = mapTop[i].genres;
            gB = mapTop[j].genres;
            
            for (g in gA) {
                if (g in gB) {
                    genre = gA[g];
                    if (genre in genresRelation) {
                        genresRelation[genre]++;     
                    } else {
                        genresRelation[genre] = -1;
                    }
                }
            }
        }
    }
}

for (i in mapTop) {
    for (j in mapTop) {
        if (i != j) {
            gA = mapTop[i].genres;
            gB = mapTop[j].genres;
            
            max = 0;
            
            e = {
                source: i,
                target: j,
                popularity: mapTop[i].popularity * mapTop[j].popularity
            }
            
            for (g in gA) {
                if (gB.find((val) => {return val === gA[g];})) {
                    genre = gA[g];
                    
                    if (genresRelation[genre] > max) {
                        max = genresRelation[genre];
                        e.type = genre;
                    }
                }
            }
            
            if(e.type) {
                graph.edges.push(e);    
            }
        }
    }
}

console.log('End of graphing');

Start of graphing
End of graphing


In [300]:
// graph is {nodes:[a,b,c...], edges:[{source:a, target:b}...]}
function mst(graph, type) {
    // Create a forest 'trees' (a set of trees), where each vertex
    // in the graph is a separate tree.
    var trees = {};
    
    for (var i = 0, c = graph.nodes.length; i < c; i++) {
        trees[graph.nodes[i].id] = makeset();
    }
    
    // Create a set 'edges' containing all the edges in the graph.
    var edges = [];
    if (type) {      
        edges = graph.edges.filter(function(val){
            return val.type == type;
        }).slice(); // Copy using slice.    
    } else {
        edges = graph.edges.slice();
    }
    
    edges.sort(function(a, b) {return b.popularity - a.popularity;});
    
    var usedEdges = [];
    
    // While 'edges' is nonempty and 'trees' is not yet spanning.
    while (edges.length > 0 && Object.keys(trees).length > 1) {
        // Remove an edge with minimum weight from 'edges'
        var edge = edges.pop();
        
        if (find(trees[edge.source]) !== find(trees[edge.target])) {
            usedEdges.push(edge);
            // If that edge connects two different trees, then add it to
            // the forest, combining two trees into a single tree.
            union(trees[edge.source], trees[edge.target]);
        }
        // Otherwise discard that edge.
    }
    return usedEdges;
}

console.log(mst(graph, 'pop').length);

12


In [308]:
limit = 200000;

topGenre = [];

sortedGenres = Object.keys(genresRelation).sort(function(a, b) {
  return genresRelation[a] < genresRelation[b];
});

i = 0;
for (g in sortedGenres) {
    topGenre.push(sortedGenres[g]);
    i++;
    if (i > limit) break;
}

topGenre.push('related')

topGenre

[ 'garage rock',
  'axe',
  'alternative metal',
  'permanent wave',
  'pop',
  'pop christmas',
  'rock',
  'adult standards',
  'dance pop',
  'modern rock',
  'hard rock',
  'neo mellow',
  'tropical',
  'latin pop',
  'latin',
  'album rock',
  'art rock',
  'bachata',
  'mpb',
  'classic rock',
  'samba',
  'irish rock',
  'alternative dance',
  'mellow gold',
  'psychedelic rock',
  'classical',
  'soft rock',
  'glam rock',
  'christmas',
  'new wave',
  'blues-rock',
  'dance rock',
  'hawaiian',
  'alternative rock',
  'reggaeton',
  'folk',
  'pagode',
  'bossa nova',
  'sertanejo universitario',
  'hollywood',
  'indie rock',
  'sertanejo',
  'brazilian punk',
  'glam metal',
  'acoustic pop',
  'heavy christmas',
  'rock-and-roll',
  'rock en espanol',
  'latin arena pop',
  'soundtrack',
  'sheffield indie',
  'soul christmas',
  'disco',
  'pop punk',
  'punk',
  'merseybeat',
  'british invasion',
  'pop rock',
  'merengue urbano',
  'piano rock',
  'post-grunge',
  'ope

In [309]:
filterEdges = [];

for (i in topGenre) {    
    tree = mst(graph, topGenre[i]);
    filterEdges = filterEdges.concat(tree);
}

/*JSON.stringify({
    "nodes": graph.nodes,
    "edges": filterEdges
});*/

filterEdges

[ { source: '57xRtfFlssbpntpRQOBodd',
    target: '2HtEBtekn0bXogyogD3PTr',
    popularity: 3660,
    type: 'axe' },
  { source: '12Chz98pHFMPJEknJQMWvI',
    target: '2jzc5TC5TVFLXQlBNiIUzE',
    popularity: 5688,
    type: 'permanent wave' },
  { source: '2jzc5TC5TVFLXQlBNiIUzE',
    target: '51Blml2LZPmy7TTiAg47vQ',
    popularity: 5760,
    type: 'permanent wave' },
  { source: '0oSGxfWSnnOXhD2fKuz2Gy',
    target: '2jzc5TC5TVFLXQlBNiIUzE',
    popularity: 5760,
    type: 'permanent wave' },
  { source: '7jy3rLJdDQY21OgRLCZ9sD',
    target: '2jzc5TC5TVFLXQlBNiIUzE',
    popularity: 5832,
    type: 'permanent wave' },
  { source: '7oPftvlwr6VrsViSDV7fJY',
    target: '2jzc5TC5TVFLXQlBNiIUzE',
    popularity: 5904,
    type: 'permanent wave' },
  { source: '7Ln80lUS6He07XvHI8qqHH',
    target: '2jzc5TC5TVFLXQlBNiIUzE',
    popularity: 6048,
    type: 'permanent wave' },
  { source: '4gzpq5DPGxSnKTe4SA8HAU',
    target: '2jzc5TC5TVFLXQlBNiIUzE',
    popularity: 6408,
    type: 'perman

In [310]:
newGraph = {nodes: graph.nodes, edges: filterEdges};

JSON.stringify(newGraph);

'{"nodes":[{"id":"2HtEBtekn0bXogyogD3PTr","name":"Victor & Leo","genres":["axe","sertanejo","sertanejo universitario"],"img":"https://i.scdn.co/image/75b4c4b42bb526055a40d0ecf2e426c256220924","url":"https://open.spotify.com/artist/2HtEBtekn0bXogyogD3PTr","popularity":60},{"id":"4gzpq5DPGxSnKTe4SA8HAU","name":"Coldplay","genres":["permanent wave","pop","pop christmas","rock"],"img":"https://i.scdn.co/image/143b0f286f76ece3a711f673d9ba00b8f499b2c0","url":"https://open.spotify.com/artist/4gzpq5DPGxSnKTe4SA8HAU","popularity":89},{"id":"2snm3HmsnDUDxbIChdl2Oj","name":"Justin Hurwitz","genres":["adult standards","hollywood","soundtrack"],"img":"https://i.scdn.co/image/e1206ba5c89c640cba7a00ca8409584b6262b0d2","url":"https://open.spotify.com/artist/2snm3HmsnDUDxbIChdl2Oj","popularity":68},{"id":"4phGZZrJZRo4ElhRtViYdl","name":"Jason Mraz","genres":["acoustic pop","neo mellow","pop","pop christmas","pop rock","singer-songwriter","viral pop"],"img":"https://i.scdn.co/image/05dc7e7d87ef98c8ebdb7

In [311]:
JSON.stringify({nodes: graph.nodes, edges: mst(newGraph)});

'{"nodes":[{"id":"2HtEBtekn0bXogyogD3PTr","name":"Victor & Leo","genres":["axe","sertanejo","sertanejo universitario"],"img":"https://i.scdn.co/image/75b4c4b42bb526055a40d0ecf2e426c256220924","url":"https://open.spotify.com/artist/2HtEBtekn0bXogyogD3PTr","popularity":60},{"id":"4gzpq5DPGxSnKTe4SA8HAU","name":"Coldplay","genres":["permanent wave","pop","pop christmas","rock"],"img":"https://i.scdn.co/image/143b0f286f76ece3a711f673d9ba00b8f499b2c0","url":"https://open.spotify.com/artist/4gzpq5DPGxSnKTe4SA8HAU","popularity":89},{"id":"2snm3HmsnDUDxbIChdl2Oj","name":"Justin Hurwitz","genres":["adult standards","hollywood","soundtrack"],"img":"https://i.scdn.co/image/e1206ba5c89c640cba7a00ca8409584b6262b0d2","url":"https://open.spotify.com/artist/2snm3HmsnDUDxbIChdl2Oj","popularity":68},{"id":"4phGZZrJZRo4ElhRtViYdl","name":"Jason Mraz","genres":["acoustic pop","neo mellow","pop","pop christmas","pop rock","singer-songwriter","viral pop"],"img":"https://i.scdn.co/image/05dc7e7d87ef98c8ebdb7